In [1]:
import pandas as pd
import nltk

In [4]:
df = pd.read_csv('coursera_sessions_train.txt', sep=';', header=None)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
0    50000 non-null object
1    3608 non-null object
dtypes: object(2)
memory usage: 781.4+ KB
